In [1]:
from transformers import BertTokenizer, BertModel

C:\Users\Jay\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\Jay\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jay\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
token=tokenizer("I love money",return_tensors='pt')
token

{'input_ids': tensor([[ 101, 1045, 2293, 2769,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

<b>input_ids: tensor([[101, 1045, 2293, 2769, 102]]) maps tokens to vocabulary IDs: 101=[CLS], 1045="i", 2293="love", 2769="money", 102=[SEP]. These integers serve as embeddings lookup indices in BERT.<br><br>
token_type_ids: tensor([[0, 0, 0, 0, 0]]) indicates segment membership (all 0s here for one sentence; used in next-sentence prediction tasks).<br><br>
attention_mask: tensor([[1, 1, 1, 1, 1]]) marks real tokens (1) versus padding (0, absent here), guiding self-attention to ignore pads in batched inputs.
</b>

In [4]:
token=tokenizer(["I love apple","Apple a days keeps a doctor away"],return_tensors='pt')
token

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

<b>The resulting token['input_ids'] (and other keys) will have ragged shapes due to unequal lengths after tokenization—"I love apple" becomes ~5 tokens, while "Apple a days keeps a doctor away" becomes ~9—requiring padding for consistent tensor dimensions in PyTorch models. BERT models expect fixed-shape inputs,Without padding=True, longer sequences truncate or raise shape mismatch errors during batching/model forward passes, Padding adds [PAD] tokens (ID 0) and sets attention_mask=0 for them, enabling efficient batched processing.
</b>

In [5]:
token=tokenizer(["I love apple","Apple a days keeps a doctor away"],padding=True,return_tensors='pt')
token

{'input_ids': tensor([[ 101, 1045, 2293, 6207,  102,    0,    0,    0,    0],
        [ 101, 6207, 1037, 2420, 7906, 1037, 3460, 2185,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]])}

<b>It is clearly understable from above output that as sentence 1 is shorter compared to sentence 2 0's were appended to end of the list and attention_mask for those appended 0 are 0 enablging efficient batch processing<b>

In [6]:
# we ccan also play with padding. example
token=tokenizer(["I love apple","Apple a days keeps a doctor away"],padding='max_length',max_length=10,return_tensors='pt')
token

{'input_ids': tensor([[ 101, 1045, 2293, 6207,  102,    0,    0,    0,    0,    0],
        [ 101, 6207, 1037, 2420, 7906, 1037, 3460, 2185,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}

<h4>Here the word length of sentence 2 is 9 where 7 are words and other 2 are starter and separator so only 1 space left so at last 1 was appended padding='max_length' force the tokens to be represented in 10 length</h4>

In [7]:
#What is padding length is less
# we ccan also play with padding. example
token=tokenizer(["I love apple","Apple a days keeps a doctor away"],padding='max_length',max_length=5,return_tensors='pt')
token

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

<h5>We can solve the above error by using truncation. This parameter allows the tokenizer to remove the tokens>max_length</h5>

In [8]:
#Same code but with truncation =True
token=tokenizer(["I love apple","Apple a days keeps a doctor away"],padding='max_length',max_length=5,truncation=True,return_tensors='pt')
token

{'input_ids': tensor([[ 101, 1045, 2293, 6207,  102],
        [ 101, 6207, 1037, 2420,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}

In [10]:
#Above 101 and 102 are started and separator rest words whose upto 3 are tokenized
#Other extra words are just truncated
#Now let's resume our work
token=tokenizer("I love money",return_tensors='pt')
# token
model=BertModel.from_pretrained('bert-base-uncased')
output=model(**token)

<b>This code tokenizes the input text "I love money" using BERT's uncased tokenizer and passes the result through a pretrained BERT-base-uncased model to generate contextual embeddings.</b>

In [11]:
output['last_hidden_state'] 

tensor([[[ 0.1643,  0.3634, -0.1257,  ..., -0.2683,  0.1275,  0.2219],
         [ 0.4908,  0.4567, -0.4418,  ..., -0.3918,  0.6911,  0.0345],
         [ 1.7736,  0.9322,  0.5129,  ..., -0.4638,  0.5791, -0.1290],
         [-0.2870, -0.6937, -0.5570,  ...,  0.2537,  0.4886, -0.3197],
         [ 0.8139,  0.1525, -0.1483,  ..., -0.0639, -0.6087, -0.2690]]],
       grad_fn=<NativeLayerNormBackward0>)

In [12]:
#for each word in the token there would be a series of contextual embedding as you can notice here
#Here there were 5 token 2 for start and space and 3 actual word
#And we got the same 2D list with 5 row each row representing each token's contextual embedding

In [14]:
#Now let's check the shape of the last hidden state
output['last_hidden_state'].shape

torch.Size([1, 5, 768])

In [17]:
tokenizer.convert_ids_to_tokens(token['input_ids'][0])
#Using above method we can print the token from its particular id

['[CLS]', 'i', 'love', 'money', '[SEP]']

In [19]:
output['pooler_output'].shape
#this is the contextual embedding for the whole sentence

torch.Size([1, 768])

<h3>Note**: You can also use CLS embedding because the CLS embedding i.e 'last_hidden_state''s first output of CLS
also represents the contextual embedding for the whole sentence differnce between pooler_output and CLS output 
is that pooler_output is generated from CLS by passing it to some Neural Network and applying TANH to it</h3>